In [1]:
import pandas as pd
import numpy as np
from data_reader import read_df

In [2]:
df = read_df('test')
df.head()

,user_id,item_id,rate,review_text
0,A35HOUWHAYZZN6,B00JF3RYPM,1,This lotion gently moisturizes without irritat...
1,A1DGB6HY5C7LXU,B00KLA4INE,1,I've been doing some research into non-surgica...
2,A3EPHBMU07LZ50,B00CBD0M8Y,1,I could not detectably notice much difference ...
3,A1WSQ0QRWQC7VI,B001FSK8SA,1,arived just in time for xmas so i used it as a...
4,A2SZLNSI5KOQJT,B00D18ZDM8,1,"In my opinion, the Bic Soleil Shave & Trim Sha..."


In [33]:
users = set(df['user_id'])
fav = {u:df[df['user_id']==u]['item_id'].tolist() for u in users}

In [38]:
u = 'A3EPHBMU07LZ50'
fav[u]

['B00CBD0M8Y', 'B00AE07FQI', 'B00AE07GX0', 'B001OMI93S']

In [41]:
golds = {u:list(enumerate(fav[u])) for u in users}

In [19]:
list(fav.values())[2]

['B00A0IWRRG',
 'B000E0K7NQ',
 'B00HB2JQNM',
 'B00AE0790U',
 'B0072XRJ20',
 'B00HD7SCLM',
 'B00GLS5DVQ',
 'B00AE07FXQ']

In [39]:
golds[u]

[(0, 'B00CBD0M8Y'), (1, 'B00AE07FQI'), (2, 'B00AE07GX0'), (3, 'B001OMI93S')]

In [45]:
def rank_correlation(user, sorting):
    gold = fav[user]
    best = sorting[:20]

    return len(set(best).intersection(gold))

def rc_test():
    user = 'A2G04D4QZAXL15'
    items = fav[user] + fav['A3JT29L4YFEIMJ']
    np.random.shuffle(items)
    i = items[0:5]

    fav[user], i, rank_correlation(user, i)

(['B00A0IWRRG',
  'B000E0K7NQ',
  'B00HB2JQNM',
  'B00AE0790U',
  'B0072XRJ20',
  'B00HD7SCLM',
  'B00GLS5DVQ',
  'B00AE07FXQ'],
 ['B00AE07FXQ', 'B000E0K7NQ', 'B005HIHD2I', 'B00HB2JQNM', 'B0098JU8T2'],
 3)

In [ ]:
def DCG(user, sorting):
    gold = golds[user]
    best = sorting[:20]